<a href="https://colab.research.google.com/github/kanru-wang/coursera_quantization_pruning_distillation/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Keras Tuner

## Download and prepare the dataset

In [ ]:
# Import keras
from tensorflow import keras

In [ ]:
# Download the dataset and split into train and test sets
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

Normalize the pixel values to make the training converge faster.

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Baseline Performance

In [ ]:
# Build the baseline model using the Sequential API
b_model = keras.Sequential()
b_model.add(keras.layers.Flatten(input_shape=(28, 28)))
b_model.add(keras.layers.Dense(units=512, activation='relu', name='dense_1')) # Will tune this layer later
b_model.add(keras.layers.Dropout(0.2))
b_model.add(keras.layers.Dense(10, activation='softmax'))

# Print model summary
b_model.summary()

In [ ]:
# Setup the training parameters
b_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001), # Will tune learning rate later
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
# Number of training epochs.
NUM_EPOCHS = 10

# Train the model
b_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

In [ ]:
# Evaluate model on the test set
b_eval_dict = b_model.evaluate(img_test, label_test, return_dict=True)

In [ ]:
# Define helper function
def print_results(model, model_name, layer_name, eval_dict):
    '''
    Prints the values of the hyparameters to tune, and the results of model evaluation

    Args:
        model (Model) - Keras model to evaluate
        model_name (string) - arbitrary string to be used in identifying the model
        layer_name (string) - name of the layer to tune
        eval_dict (dict) -  results of model.evaluate
    '''
    print(f'\n{model_name}:')

    print(f'number of units in 1st Dense layer: {model.get_layer(layer_name).units}')
    print(f'learning rate for the optimizer: {model.optimizer.lr.numpy()}')

    for key,value in eval_dict.items():
        print(f'{key}: {value}')

# Print results for baseline model
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)

## Keras Tuner

To perform hypertuning with Keras Tuner, need to:

* Define the model
* Select which hyperparameters to tune
* Define the search space
* Define the search strategy

### Install and import packages

In [ ]:
# Install Keras Tuner
!pip install -q -U keras-tuner

In [ ]:
# Import required packages
import tensorflow as tf
import keras_tuner as kt

### Define the model

The model for hypertuning is called a *hypermodel*. Need to define the hyperparameter search space in addition to the model architecture. 

Two approaches to define a hypermodel:

* By using a model builder function
* By [subclassing the HyperModel class](https://keras-team.github.io/keras-tuner/#you-can-use-a-hypermodel-subclass-instead-of-a-model-building-function) of the Keras Tuner API


In below we use the first approach: Use a model builder function to define the image classification model. This function returns a compiled model and uses hyperparameters defined inline to hypertune the model.

Two hyperparameters that are setup for tuning:

* the number of hidden units of the first Dense layer
* the learning rate of the Adam optimizer

HyperParameters object configures the hyperparameter:

* use `Int()` to define the search space for the Dense units

* use `Choice()` for the learning rate

In [ ]:
def model_builder(hp):
    '''
    Builds the model and sets up the hyperparameters to tune.

    Args:
        hp - Keras tuner object

    Returns:
        model with hyperparameters to tune
    '''
    
    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', name='tuned_dense_1'))

    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

## Instantiate the Tuner and perform hypertuning

Keras Tuner has four tuners available with built-in strategies - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. 

Here we use the Hyperband tuner. Similar to sport championship, the algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round.

Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

The `directory` save logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.

In [ ]:
# Instantiate the tuner
tuner = kt.Hyperband(
    model_builder, # the hypermodel
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='kt_dir',
    project_name='kt_hyperband'
)

In [ ]:
# Display hypertuning settings
tuner.search_space_summary()

In [ ]:
# Pass in an EarlyStopping callback to stop training early when a metric is not improving
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Perform hypertuning
tuner.search(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

## Build and train the model

Now that you have the best set of hyperparameters, you can rebuild the hypermodel with these values and retrain it.

In [ ]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

In [ ]:
# Train the hypertuned model
h_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

In [ ]:
# Evaluate the hypertuned model against the test set
h_eval_dict = h_model.evaluate(img_test, label_test, return_dict=True)

In [ ]:
# Print results of the baseline and hypertuned model
print_results(b_model, 'BASELINE MODEL', 'dense_1', b_eval_dict)
print_results(h_model, 'HYPERTUNED MODEL', 'tuned_dense_1', h_eval_dict)